In [3]:
import os
import openai
from env_config import Config

openai.api_key = Config.OPENAI_API_KEY

# message = ["hi", "i have a cat"]
# for i in range(len(message)):
#     completion = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "user", "content": message[i]}
#     ],
#     n = 1
#     )
#     print(completion)



In [7]:
system_message = "You are a PaulGrahamGPT \n Below is a informational summary about Paul Graham life \n"

file_path = r"C:\Users\DELL\Documents\python_projects\llm_tech\data\paul_graham\paul_graham_essay.txt"

with open(file_path) as f:
    text = f.read()
    system_message += text
    system_message = system_message.split('Notes')[0]

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": "When did Paul Graham's mother has the stroke? And what happened after that?"}
    ],
    )

print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Paul Graham's mother had a stroke in the summer of 2012. As a result of the stroke, her balance was destroyed, and she had to be placed in a nursing home. However, Paul and his sister were determined to help her get out and return to her house. Paul frequently flew to Oregon to visit her, and during these visits, he had a lot of time to reflect.\n\nUnfortunately, his mother's cancer had returned. The cancer was the cause of the blood clot that led to her stroke. Sadly, she passed away on January 15, 2014. This was a difficult time for Paul and his family, as losing a loved one is always painful.\n\nFollowing his mother's passing, Paul decided to step back from his work at Y Combinator, the startup accelerator he co-founded. He wanted to take some time for himself and explore new interests. He began painting, which had always been a passion of his, and started writing essays ag

In [ ]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    print(response)
    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )
        print(function_response)

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

In [2]:
from serpapi import GoogleSearch

params = {
      "engine": "google",
      "q": "weather in delhi today",
      "api_key": Config.SERP_API_KEY
    }

search = GoogleSearch(params)
results = search.get_dict()
print(results.keys())

# organic_results = results["answer_box"]
# print(organic_results.keys())

# organic_results[:2]

NameError: name 'Config' is not defined

In [22]:
import openai
import json
from serpapi import GoogleSearch

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def search_google(query):
    params = {
      "engine": "google",
      "q": query,
      "api_key": Config.SERP_API_KEY
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["knowledge_graph"]

    return organic_results

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What is the weather in the Delhi today?"}],
        functions=[
            {
                "name": "search_google",
                "description": "Use this function to Search on google to provide accurate answers for current affairs",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "query to search on google search engine",
                        },
                    },
                    "required": ["query"],
                },
            }
        ],
        function_call="auto",
    )
    print(response)
    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        query = eval(message["function_call"]['arguments'])

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = search_google(query['query'])
        print(function_response[:2])

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather forcast for Delhi today?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response[:2],
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"query\": \"weather in Delhi today\"\n}",
          "name": "search_google"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686737215,
  "id": "chatcmpl-7RHgtEux7goEdA6ooKMYk7VQURZOK",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 72,
    "total_tokens": 90
  }
}
[{'position': 1, 'title': 'Delhi, Delhi, India Weather Forecast', 'link': 'https://www.accuweather.com/en/in/delhi/202396/weather-forecast/202396', 'displayed_link': 'https://www.accuweather.com › delhi › weather-forecast', 'favicon': 'https://serpapi.com/searches/64899012b7b1ccfb6fcd7b46/images/50deac332d20c7a50ecd15fe4cc40be05dfc87098aac3abdf29b52ff7125f383.png', 'snippet': 'Delhi, Delhi, India Weather Forecast, with current conditions, wind, 